In [132]:
from gensim.models import KeyedVectors
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, LSTM, TimeDistributed, Activation
import keras_metrics

import numpy as np

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [136]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [269]:
def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [17]:
cn_file = './data/numberbatch-en-17.06.txt'

cn_vectors = KeyedVectors.load_word2vec_format(cn_file, binary=False)

In [6]:
def split_tags(string):
    return [tuple(i.split("/")) for i in string.split()]

def readTrainData(filename):
    data = []
    for line in open(filename):
        line = line.strip()
        #read in training or dev data with labels
        if len(line.split('\t')) == 7:
            (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = \
            line.split('\t')
        else:
            continue
        # ignoring the training data that has middle label 
        nYes = eval(judge)[0]            
        if nYes >= 3:
            amt_label = True
            data.append((split_tags(origsenttag), split_tags(candsenttag), amt_label))
        elif nYes <= 1:
            amt_label = False
            data.append((split_tags(origsenttag), split_tags(candsenttag), amt_label))
    return data

def readTestData(filename):
    data = []
    for line in open(filename):
        line = line.strip()
        #read in training or dev data with labels
        if len(line.split('\t')) == 7:
            (trendid, trendname, origsent, candsent, judge, origsenttag, candsenttag) = \
            line.split('\t')
        else:
            continue
        # ignoring the training data that has middle label 
        nYes = int(judge[0])
        if nYes >= 4:
            expert_label = True
        elif nYes <= 2:
            expert_label = False
        else:
            expert_label = None
        data.append((split_tags(origsenttag), split_tags(candsenttag), expert_label))
    return data

In [386]:
dev_data = readTrainData("./data/data/dev.data")
train_data = readTrainData("./data/data/train.data")
test_data = readTestData("./data/data/test.data")

len(dev_data)  #4142
len(test_data)  #972
len(train_data)  #11530

11530

In [390]:
def getTweetEmbedding(tweet):
    word_vectors = []
    exceptions = 0
    for word in tweet:
        try:
            word_vector = cn_vectors.get_vector(word[0].lower())
        except:
            continue
        word_vectors.append(word_vector)
        
        
    return np.mean(np.array(word_vectors), axis=0), exceptions


def getLabel(label):
    if label == True:
        return 1
    if label == False:
        return 0
    else:
        return None
    

def getLabelsFeatures(data, is_train):
    labels, features = [], []
    for row in data:
        lbl = getLabel(row[2])
        if lbl is None and is_train:
            continue
        labels.append(lbl)
        embedding, _ = getTweetEmbedding(row[1])
        features.append(embedding)
    return labels, features
    
    
dev_labels, dev_features = getLabelsFeatures(dev_data, is_train=True)
test_labels, test_features = getLabelsFeatures(test_data, is_train=False)
train_labels, train_features = getLabelsFeatures(train_data, is_train=True)


dev_features = np.array(dev_features)
dev_labels = np.array(dev_labels)

test_features = np.array(test_features)
test_labels = np.array(test_labels)

train_features = np.array(train_features)
train_labels = np.array(train_labels)

*** KERAS model

In [359]:
model = Sequential()

model.add(Dense(512, activation='relu', input_dim=300))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[precision, recall, f1])

In [360]:
loss_and_metrics = model.fit(train_features, train_labels, epochs=10, batch_size=50)

print(model.metrics_names)
print(loss_and_metrics)

Epoch 1/10
11530/11530 [==============================] - 5s 397us/step - loss: 0.5536 - precision: 0.6072 - recall: 0.3807 - f1: 0.4396
Epoch 2/10
11530/11530 [==============================] - 2s 154us/step - loss: 0.4771 - precision: 0.7489 - recall: 0.5721 - f1: 0.6354
Epoch 3/10
11530/11530 [==============================] - 2s 159us/step - loss: 0.4387 - precision: 0.7963 - recall: 0.6308 - f1: 0.6892
Epoch 4/10
11530/11530 [==============================] - 2s 159us/step - loss: 0.4012 - precision: 0.8074 - recall: 0.6642 - f1: 0.7188
Epoch 5/10
11530/11530 [==============================] - 2s 179us/step - loss: 0.3770 - precision: 0.8166 - recall: 0.6937 - f1: 0.7402
Epoch 6/10
11530/11530 [==============================] - 2s 204us/step - loss: 0.3508 - precision: 0.8339 - recall: 0.7160 - f1: 0.7627
Epoch 7/10
11530/11530 [==============================] - 2s 216us/step - loss: 0.3321 - precision: 0.8391 - recall: 0.7417 - f1: 0.7800
Epoch 8/10
11530/11530 [=================

In [361]:
loss_and_metrics = model.evaluate(test_features, test_labels, batch_size=50)

print(model.metrics_names)
print(loss_and_metrics)

# ['loss', 'precision', 'recall', 'f1']
# [0.7009349179082952, 0.21241437910493632, 0.1699266110620237, 0.13016103533472254]

838/838 [==============================] - 1s 1ms/step
['loss', 'precision', 'recall', 'f1']
[0.7009349179082952, 0.21241437910493632, 0.1699266110620237, 0.13016103533472254]


In [297]:
predictions = model.predict(test_features)

In [ ]:
for prediction in predictions:
    if prediction >= 0.5:
        print('true\t' + "{0:.4f}".format(prediction[0]))
    else:
        print('false\t' + "{0:.4f}".format(prediction[0]))

*** Simple classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# clf = DecisionTreeClassifier(max_depth=10)
# 'test_f1_macro': array([0.54490802, 0.56093429, 0.56139427])

# clf = RandomForestClassifier(max_depth=10, n_estimators=10, max_features=5)
#'test_f1_macro': array([0.51949302, 0.51380182, 0.53575825])

# clf = SVC(gamma='auto', random_state=42)
# 'test_f1_macro': array([0.39521712, 0.39518414, 0.39518414]),

cross_validate(clf, train_features, train_labels, scoring=['recall_macro', 'precision_macro', 'f1_macro'], cv=3, verbose=True)

In [392]:
clf = DecisionTreeClassifier(max_depth=10)
clf.fit(train_features, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [393]:
predictions = clf.predict_proba(test_features)

In [394]:
for prediction in predictions:
    label = np.argmax(prediction)
    if prediction[1] > 0.5:
        print('true\t' + "{0:.4f}".format(prediction[1]))
    else:
        print('false\t' + "{0:.4f}".format(prediction[1]))

true	1.0000
false	0.1274
true	0.7185
false	0.0800
false	0.1616
false	0.1274
false	0.3869
true	1.0000
false	0.4268
true	1.0000
true	1.0000
false	0.1274
false	0.1199
false	0.2747
false	0.4783
false	0.2747
true	1.0000
false	0.2400
false	0.1274
true	1.0000
false	0.2400
false	0.4783
false	0.1274
true	0.8824
false	0.3869
false	0.4783
false	0.2400
true	0.7185
false	0.1274
false	0.0250
false	0.1067
false	0.1468
false	0.1199
false	0.1199
false	0.1199
false	0.1199
false	0.1199
false	0.0778
true	0.9167
true	0.7273
false	0.2400
false	0.0911
false	0.1199
true	0.8992
false	0.1199
true	0.9286
false	0.1199
false	0.0294
false	0.1199
false	0.1274
false	0.1199
false	0.0388
false	0.1199
false	0.1199
false	0.1199
true	1.0000
false	0.1451
false	0.1199
false	0.2008
false	0.3103
true	1.0000
false	0.3400
false	0.1199
true	1.0000
false	0.0800
true	0.7185
true	0.6087
true	1.0000
false	0.2008
true	0.7185
false	0.0800
false	0.1274
false	0.0000
true	0.7185
false	0.2747
false	0.0000
false	0.0147
false	0.0778
false	0